In [7]:
from sympy.physics.quantum.qapply import qapply
from sympy.physics.quantum.qubit import Qubit, matrix_to_qubit
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum.gate import HadamardGate
from sympy.physics.quantum.represent import represent
from sympy import sqrt

from oracle import oracle

In [8]:
def deutsch(f):
    """
    Run Deutsch algorithm and return |x>*|y XOR f(x)>
    :param func f: oracle function

    The problem:
    Given a function f {0, 1} -> {0, 1} that we can evaluate
    but is unknown to us determine if it is balanced or
    constant
    The oracle takes as input Qubits |x> and|y>
    and returns |x, y XOR f(x)>

    The Solution:
                     +-----------------+
      |x>  |0>---H---|                 |---H--M |x>
                     |       U_f       |
      |y>  |1>---H---|                 |------   |y XOR f(x)>
                     +-----------------+
    Constant zero -> +|0>*|->
    Constant  one -> -|0>*|->
    Balanced 0->1 -> +|1>*|->
    Balanced 1->0 -> -|1>*|->

    """
    # apply H gate to both inputs
    x = qapply(HadamardGate(0) * Qubit(0))
    print(f"|x>={x}")
    y = qapply(HadamardGate(0) * Qubit(1))
    print(f"|y>={y}")

    # calculate the Tensor Product of the inputs
    xy = TensorProduct(x, y)
    xy = matrix_to_qubit(represent(xy))
    print(f"|xy>={xy}")

    # apply oracle
    r = oracle(x, y, f)

    # apply H on top Qubit
    return qapply(HadamardGate(1) * r)


In [9]:
def test_0():

    def f(x, *args):
        return 0

    r = deutsch(f)
    print(f'\nResult const zero function: {r}')

    truth = TensorProduct(Qubit(0), ((Qubit(0) - Qubit(1)) / sqrt(2)))
    truth = matrix_to_qubit(represent(truth))
    assert truth == r

test_0()

|x>=sqrt(2)*|0>/2 + sqrt(2)*|1>/2
|y>=sqrt(2)*|0>/2 - sqrt(2)*|1>/2
|xy>=|00>/2 - |01>/2 + |10>/2 - |11>/2

Result const zero function: sqrt(2)*|00>/2 - sqrt(2)*|01>/2


In [10]:
def test_1():
    print()

    def f(x, *args):
        return 1

    r = deutsch(f)
    print(f'\nResult const one function: {r}')

    truth = TensorProduct(-Qubit(0), ((Qubit(0) - Qubit(1)) / sqrt(2)))
    truth = matrix_to_qubit(represent(truth))
    assert truth == r

test_1()


|x>=sqrt(2)*|0>/2 + sqrt(2)*|1>/2
|y>=sqrt(2)*|0>/2 - sqrt(2)*|1>/2
|xy>=|00>/2 - |01>/2 + |10>/2 - |11>/2

Result const one function: -sqrt(2)*|00>/2 + sqrt(2)*|01>/2


In [11]:
#def test_f_balanced_0():
def test_2():

    def f(x, *args):
        if x[0] == 0:
            return 1
        else:
            return 0

    r = deutsch(f)
    print(f'\nResult balanced zero function: {r}')

    truth = TensorProduct(-Qubit(1), ((Qubit(0) - Qubit(1)) / sqrt(2)))
    truth = matrix_to_qubit(represent(truth))
    assert truth == r

test_2()

|x>=sqrt(2)*|0>/2 + sqrt(2)*|1>/2
|y>=sqrt(2)*|0>/2 - sqrt(2)*|1>/2
|xy>=|00>/2 - |01>/2 + |10>/2 - |11>/2

Result balanced zero function: -sqrt(2)*|10>/2 + sqrt(2)*|11>/2


In [12]:
#def test_f_balanced_1():
def test_3():

    def f(x, *args):
        if x[0] == 0:
            return 0
        else:
            return 1

    r = deutsch(f)
    print(f'\nResult balanced one function: {r}')

    truth = TensorProduct(Qubit(1), ((Qubit(0) - Qubit(1)) / sqrt(2)))
    truth = matrix_to_qubit(represent(truth))
    assert truth == r

test_3()

|x>=sqrt(2)*|0>/2 + sqrt(2)*|1>/2
|y>=sqrt(2)*|0>/2 - sqrt(2)*|1>/2
|xy>=|00>/2 - |01>/2 + |10>/2 - |11>/2

Result balanced one function: sqrt(2)*|10>/2 - sqrt(2)*|11>/2
